In [1]:
import pandas as pd
import glob, os, json
import datetime
from datetime import datetime
import matplotlib as plt
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [2]:
# Extract calorie data from json
json_dir = 'JSON Files/Physical Activity/Calories'

json_pattern = os.path.join(json_dir, '*.json')
file_list = glob.glob(json_pattern)

dfs = []
for file in file_list:
    with open(file) as f:
        json_data = pd.json_normalize(json.loads(f.read()))
        json_data['site'] = file.rsplit("/", 1)[-1]
    dfs.append(json_data)
calorie_df = pd.concat(dfs)

In [3]:
# Convert calories to float
calorie_df['value'] = calorie_df['value'].astype(float)

In [ ]:
calorie_df

In [ ]:
# Extract date and time information
calorie_df['Date'] = pd.DatetimeIndex(calorie_df['dateTime']).date
calorie_df['Time'] = pd.DatetimeIndex(calorie_df['dateTime']).time
calorie_df['Weekday'] = pd.DatetimeIndex(calorie_df['dateTime']).weekday
calorie_df

In [ ]:
# Groupby total calories by date
weekday_calories_df = calorie_df.groupby(['Date'])['value'].sum().to_frame().reset_index()
weekday_calories_df['Weekday'] = pd.DatetimeIndex(weekday_calories_df['Date']).weekday
weekday_calories_df

In [ ]:
# Import sleep data
json_dir = 'JSON Files/Sleep'

json_pattern = os.path.join(json_dir, 'sleep*.json')
file_list = glob.glob(json_pattern)

dfs = []
for file in file_list:
    with open(file) as f:
        json_data = pd.json_normalize(json.loads(f.read()))
        json_data['site'] = file.rsplit("/", 1)[-1]
    dfs.append(json_data)
fitbit_df = pd.concat(dfs)

In [ ]:
# Add date, time, weekday
fitbit_df['wakeDate'] = pd.DatetimeIndex(fitbit_df['endTime']).date
fitbit_df['sleepDate'] = pd.DatetimeIndex(fitbit_df['startTime']).date
fitbit_df['wakeTime'] = pd.DatetimeIndex(fitbit_df['endTime']).time
fitbit_df['sleepTime'] = pd.DatetimeIndex(fitbit_df['startTime']).time
fitbit_df['Weekday'] = pd.DatetimeIndex(fitbit_df['endTime']).weekday
fitbit_df['hoursAsleep'] = (fitbit_df['minutesAsleep'] / 60)
fitbit_df

In [ ]:
# Import sleep score table
sleep_score_df = pd.read_csv('JSON Files/Sleep/sleep_score.csv')
sleep_score_df['Date'] = pd.DatetimeIndex(sleep_score_df['timestamp']).date
sleep_score_df['Time'] = pd.DatetimeIndex(sleep_score_df['timestamp']).time
sleep_score_df['Weekday'] = pd.DatetimeIndex(sleep_score_df['timestamp']).weekday
sleep_score_df['Month'] = pd.DatetimeIndex(sleep_score_df['timestamp']).month
sleep_score_df

In [ ]:
sleep_times_df = fitbit_df[['logId', 'dateOfSleep', 'minutesAsleep', 'levels.summary.deep.minutes', 'levels.summary.wake.minutes', 'levels.summary.light.minutes', 'levels.summary.rem.minutes']]
sleep_times_df = sleep_times_df.set_index('logId')
sleep_times_df[['minutesAsleep', 'levels.summary.deep.minutes', 'levels.summary.wake.minutes', 'levels.summary.light.minutes', 'levels.summary.rem.minutes']] = sleep_times_df[['minutesAsleep', 'levels.summary.deep.minutes', 'levels.summary.wake.minutes', 'levels.summary.light.minutes', 'levels.summary.rem.minutes']] / 60
sleep_times_df = sleep_times_df.rename(columns={'dateOfSleep':'Sleep_Date', 'minutesAsleep':'Hours Asleep', 'levels.summary.deep.minutes':'Deep Sleep', 'levels.summary.wake.minutes':'Awake', 'levels.summary.light.minutes':'Light Sleep', 'levels.summary.rem.minutes':'REM'})
sleep_times_df.reset_index(inplace=True)